**참고 사이트**<br>
[인스타 크롤링(1)](https://blog.naver.com/cygnet3rd/222885102852),
[인스타 크롤링(2)](https://velog.io/@leeug9002/python-selenium%EC%9D%B8%EC%8A%A4%ED%83%80-%ED%81%AC%EB%A1%A4%EB%A7%81),
[인스타 게시물 내 사진 크롤링](https://proefforter.tistory.com/36),
[리스트 중복 제거&순서 유지](https://m31phy.tistory.com/130)

**필요한 패키지 호출**

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

import requests
from re import escape
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import urllib.request
import time
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup, NavigableString, Tag

**데이터 수집하기**
- 아이디, 본문 내용, 해시태그, 작성일자, 좋아요 수, 이미지 URL, 댓글

In [2]:
def get_content(driver):
    url = driver.current_url
    driver.get(url)
    time.sleep(3)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    #아이디
    ids = soup.select('span._ap3a._aaco._aacw._aacx._aad7._aade')[0].text

    #본문 내용
    try:
        #다른 유저 태그되어있을 시 [0].text가 "및"임
        soup_select = soup.select('span.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.xt0psk2.x1i0vuye.xvs91rp.xo1l8bm.x5n08af.x10wh9bi.x1wdrske.x8viiok.x18hxmgj')
        select_cnt = len(soup_select)
        content_all = soup_select[select_cnt-1].text
        content = re.sub(r'#\S+', '', content_all).strip()
    except:
        content = ''    #본문 내용 없을 시

    #해시태그
    tags = re.findall(r'#[^\s#,\\]+', content_all)

    #작성일자
    date = soup.select('time.x1p4m5qa')[0]['datetime'][:10]

    #좋아요
    try:
        like = soup.select('span.html-span.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1hl2dhg.x16tdsg8.x1vvkbs')[0].text
    except:
        like = 0    #좋아요 없을 시
    
    #이미지
    try:
        # 이미지 객체 저장 변수 및 url저장 변수 선언
        img_urls = []
        images = []

        # 이미지 객체 찾기
        images.append(driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3"))

        try:
            while(1):
                # 다음 버튼 클릭 0.5초 지연시간 설정
                time.sleep(0.5)
                # 다음 버튼이 화면에 존재할 경우, 다음 버튼 클릭후 다음차례의 이미지 저장
                driver.find_element(By.CLASS_NAME,"_afxw._al46._al47").click()
                images.append(driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3"))
                
                # 수집한 이미지 객체 중, 3번째 객체까지 이미지 소스주소를 수집한다.
                for i in range(images.__len__()):
                    for j in range(images[i].__len__()):
                        if(j>=3):
                            break
                        img_urls.append(images[i][j].get_attribute('src'))

                # 수집한 이미지 객체 자료들 비우기
                images.clear()
        except:
            # 다음버튼이 존재하지 않을 경우
            # 리스트 - 셋 - 리스트 형변환으로 중복 데이터 삭제
            img_urls = list(dict.fromkeys(img_urls))

        del img_urls[2] #가장 최근 게시물의 썸네일 링크는 삭제

    except:
        img_urls = ''   #이미지 없을 시

    #댓글
    comments = []

    for a_tag in soup.select('span > a.notranslate._a6hd'):
        inside_text = a_tag.get_text(strip=True)
        
        # 다음 형제 노드 찾기
        next_node = a_tag.next_sibling

        # 텍스트가 바로 있을 수도, 태그가 있을 수도 있음
        outside_text = ''
        
        # 만약 next_node가 텍스트 노드면
        if isinstance(next_node, NavigableString):
            outside_text = next_node.strip()
        # 만약 다음 노드가 태그면 그 안의 텍스트 가져오기
        elif isinstance(next_node, Tag):
            outside_text = next_node.get_text(strip=True)
        
        #print(f"내부텍스트: {inside_text}, 바깥텍스트: {outside_text}")
        cmt = inside_text + ' ' + outside_text
        comments.append(cmt)

    data = [ids, content, tags, date, like, img_urls, comments]
    return data

**크롬 브라우저 열고 인스타그램 접속**

In [3]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = 'https://www.instagram.com'

driver.get(url)

**인스타그램 로그인**
- `By.NAME`: HTML 태그 중 `name='...'` 속성을 기준으로 찾겠다
- ex. `<input t ype='password' name='password'>`

In [4]:
userId = 'user id'
input_id = driver.find_element(By.NAME, 'username')
input_id.clear()
input_id.send_keys(userId)

userPw = 'user password'
input_pw = driver.find_element(By.NAME, 'password')
input_pw.clear()
input_pw.send_keys(userPw)
input_pw.submit()

**원하는 검색어(해시태그)를 입력받아 해당 검색결과 페이지로 링크**
- 페이지가 로드되는 데 시간이 소요될 수 있어 3초의 시간 지연 부여

In [5]:
word = input('검색어를 입력하세요: ')   #'연남동맛집' 검색
url = 'https://www.instagram.com/explore/search/keyword/?q=%23' + str(word)
driver.get(url)
time.sleep(3)

In [6]:
driver.find_elements(By.CSS_SELECTOR, 'div._aagw')[0].click()   #첫 번째 게시물 클릭
time.sleep(3)

post_url = []
post_url.append(driver.current_url) #현재(첫 번째) 게시물 url

def move_next(driver):
    nxt = driver.find_element(By.CSS_SELECTOR, 'div._aaqg._aaqh')
    nxt.click()   #다음 게시물로 이동
    time.sleep(3)
    now_url = driver.current_url    #이동한 게시물 url
    post_url.append(now_url)

target = 9  #수집할 게시물 개수

for i in range(target):
    try:
        move_next(driver)   #target만큼 게시물의 url 모으기
        time.sleep(3)
    except:
        time.sleep(2)
        move_next(driver)
        time.sleep(3)

In [ ]:
results = []    #수집한 데이터값 담을 리스트

for post in post_url:
    url = post
    driver.get(url) #게시물 url로 이동

    data = get_content(driver)  #크롤링
    results.append(data)
    time.sleep(3)

len(results)

In [ ]:
results_df = pd.DataFrame(results)
results_df.columns = ['id', 'content', 'tags', 'date', 'like', 'img_urls']

results_df

#해시태그가 댓글에 있는 경우도 있음